# Drive

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Create and move file in drive

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. This query shows you how to create and move files.
-- 2. Files are created using the Drive.SaveAs function.
-- 3. Files are moved using the Drive.File.Operation provider
-- More details on using Luminesce with Drive here:
-- https://support.lusid.com/knowledgebase/article/KA-01688/en-us
-- ============================================================

-- Define a file and folder names

@@file_name = select 'daily_instruments_20220215';
@@create_file_location = select '/luminesce-examples/new/';
@@new_file_location = select '/luminesce-examples/archive/';

-- Collect sample instruments from the Lusid.Instrument provider

@instruments = select * from Lusid.Instrument limit 10;

-- Save the instruments to Drive as a CSV

@save_to_drive = use Drive.SaveAs with @instruments, @@file_name, @@create_file_location
--path=/{@@create_file_location}
--fileNames={@@file_name}
enduse;

-- Move the instruments CSV file to a new location
-- We need to add the wait statement, so Luminesce knows to finish the file writer,
-- before attempting to move the file

@move_file_request =
select (@@create_file_location || @@file_name || '.csv' ) as [FullPath],
(@@new_file_location || @@file_name || '.csv') as [NewFullPath],
'MoveRenameMayOverwrite' as [Operation] wait 2;

select * from Drive.File.Operation wait 5
where UpdatesToPerform = @move_file_request;




#### Create log file on error

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we create a table of two transactions.
-- 2. These transactions are saved to LUSID Drive.
-- ============================================================

-- Define some variables for file and folder name

@@file_name = select 'daily_transactions_20220215.csv';
@@folder_name = select '/luminesce-examples/new';

-- populate a variable with a table of transactions

@transactions = 
select "GB" as ClientInternal,
"Test" as PortfolioCode,
"IborTest" as PortfolioScope, 
"GBP" as TradeCurrency,
"GBP" as SettlementCurrency,
#2022-01-01# as TradeDate,
#2022-01-03#  as SettlementDate,
100 as TradePrice,
"trd_001" as TxnId, 
"StockIn" as Type,
10 as Units
union
select
"GB" as ClientInternal,
"Test" as PortfolioCode,
"IborTest" as PortfolioScope, 
"GBP" as TradeCurrency,
"GBP" as SettlementCurrency,
#2022-01-01# as TradeDate,
#2022-01-03#  as SettlementDate,
100 as TradePrice,
"trd_002" as TxnId, 
"StockIn" as Type,
10 as Units;

-- Write the transaction table to LUSID Drive
-- You can pass in the @@file_name and @@folder_name as variables.
-- You can also pass the file_name as folder name as strings

@save_to_drive = use Drive.SaveAs with @transactions, @@file_name, @@folder_name
--path=/{@@folder_name}
--fileNames={@@file_name}
enduse;

select * from @save_to_drive;


#### Read a file from drive

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- 1. This query shows you how to load a CSV file from LUSID Drive
-- ===============================================================

@data = use Drive.csv
--file=/luminesce-examples/equity_transactions.csv
enduse;

select * from @data;



#### Read an excel file from drive

In [ ]:
%%luminesce

-- ================================================================
-- Description:
-- This query shows you how to load an Excel file from LUSID Drive
-- We use the worksheet parameter to load a specific worksheet
-- See further details on the Drive.Excel provider here:
-- https://support.lusid.com/knowledgebase/article/KA-01682/en-us
-- ================================================================

@@worksheet = select 'holdings';

@holdings = use Drive.Excel with @@worksheet
--file=/luminesce-examples/equity_holding.xlsx
--worksheet={@@worksheet}
enduse;

select * from @holdings;



#### Save data into drive

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we create a table of two transactions.
-- 2. These transactions are saved to LUSID Drive.
-- ============================================================

-- Define some variables for file and folder name

@@file_name = select 'daily_transactions_20220215.csv';
@@folder_name = select '/luminesce-examples/new';

-- populate a variable with a table of transactions

@transactions =
select "GB" as ClientInternal,
"Test" as PortfolioCode,
"IborTest" as PortfolioScope, 
"GBP" as TradeCurrency,
"GBP" as SettlementCurrency,
#2022-01-01# as TradeDate,
#2022-01-03# as SettlementDate,
100 as TradePrice,
"trd_001" as TxnId, 
"StockIn" as Type,
10 as Units
union
select
"GB" as ClientInternal,
"Test" as PortfolioCode,
"IborTest" as PortfolioScope, 
"GBP" as TradeCurrency,
"GBP" as SettlementCurrency,
#2022-01-01# as TradeDate,
#2022-01-03#  as SettlementDate,
100 as TradePrice,
"trd_002" as TxnId, 
"StockIn" as Type,
10 as Units;

-- Write the transaction table to LUSID Drive
-- You can pass in the @@file_name and @@folder_name as variables.
-- You can also pass the file_name as folder name as strings

@save_to_drive = use Drive.SaveAs with @transactions, @@file_name, @@folder_name
--path=/{@@folder_name}
--fileNames={@@file_name}
enduse;

select * from @save_to_drive;


#### Unformatted string to table

In [ ]:
%%luminesce

-- Description:
-- In this example we load raw data from a 
-- txt file into a Luminesce table. The 
-- data is in a comma separated format

/*
====================================================
    1. Load comma separated string from txt file
====================================================
*/

@@txtDataOriginal = use Drive.RawText
--file=/luminesce-examples/prices-in-txt-file.txt
enduse;

@@txtData_RemoveRedundantTxt = select regexp_replace(@@txtDataOriginal, '"|ADD| ', '');

@@txtData_CommaSeparatedString = select regexp_match(@@txtData_RemoveRedundantTxt, '(?<=ENDRECORD)[.|\n|\W|\w]*');

/*
======================================
    2. Parse out the columns names 
======================================
*/

@@columnsWithNoSepCommas = select regexp_match(@@txtDataOriginal , '(?<=RECORD)([.|\n|\W|\w]*)(?=ENDRECORD)');

@@columnsWithTrailingComma = select regexp_replace(@@columnsWithNoSepCommas, '\r\n', ',');

@columnsWithSepCommas = select substr(@@columnsWithTrailingComma, 2, length(@@columnsWithTrailingComma) -2) as ColumnNames;

/*
==========================================
    3. Split string into separate rows
==========================================
*/

-- Note: This is how you split by endlines in Luminesce

@txtSplitByRow = select Value from Tools.Split where DelimiterString = '
'
and Original = @@txtData_CommaSeparatedString;

@tableRowsAsOneCol = select distinct Value as OriginalColumnValue from @txtSplitByRow
where Value != '';

/*
====================================================================    
    4. Use the Tools.Split provider to split the data into cells    
====================================================================
*/

-- Split the table body first

@tableRowsAsOneColWithIndex = select r.OriginalColumnValue, t.*
from @tableRowsAsOneCol r
inner join Tools.Split t 
on t.Original = r.OriginalColumnValue;

-- Then split column names

@columnsNamesToTable = select r.ColumnNames, t.*
from @columnsWithSepCommas r
inner join Tools.Split t 
on t.Original = r.ColumnNames;

-- Join the table body and column names together

@formattedTable = select 
tr.Value as [RowValue], tr.[OriginalIndex], ct.Value as [ColumnName]
from @columnsNamesToTable ct
join @tableRowsAsOneColWithIndex tr on (ct.[Index] = tr.[Index])
;

/*
=======================================
    5. Pivot and format the results
=======================================
*/

@pivoted = 
use Tools.Pivot with @formattedTable
--key=ColumnName
--aggregateColumns=RowValue
enduse;

select
*
from @pivoted;

